In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

# Sử dụng ChromeDriverManager để tự động tải và khởi tạo ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# URL của trang danh sách điện thoại trên Thế Giới Di Động
url = 'https://www.thegioididong.com/laptop'

# Danh sách lưu trữ dữ liệu sản phẩm
all_products_data = []

try:
    # Truy cập trang web
    print(f"Truy cập trang: {url}")
    driver.get(url)

    # Chờ danh sách sản phẩm xuất hiện
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'listproduct'))
    )

    # Nhấn nút "Xem thêm" cho đến khi không còn nút này
    # Nhấn nút "Xem thêm" cho đến khi không còn nút này
    print("Nhấn nút 'Xem thêm' để tải hết sản phẩm...")
    max_attempts = 3  # Số lần thử lại tối đa nếu gặp lỗi
    attempts = 0

    while True:
        try:
            # Lưu số lượng sản phẩm hiện tại
            product_list = driver.find_element(By.CLASS_NAME, 'listproduct')
            current_product_count = len(product_list.find_elements(By.TAG_NAME, 'li'))

            # Kiểm tra xem nút "Xem thêm" có tồn tại không
            view_more_buttons = driver.find_elements(By.CLASS_NAME, 'see-more-btn')
            if not view_more_buttons:
                print("Không tìm thấy nút 'Xem thêm', đã tải hết sản phẩm.")
                break

            # Lấy nút "Xem thêm" đầu tiên
            view_more_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'see-more-btn'))
            )

            # Cuộn đến nút "Xem thêm"
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", view_more_button)
            time.sleep(1)  # Đợi cuộn hoàn tất

            # Nhấn nút
            view_more_button.click()
            print("Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...")

            # Chờ danh sách sản phẩm cập nhật
            WebDriverWait(driver, 30).until(
                lambda d: len(d.find_element(By.CLASS_NAME, 'listproduct').find_elements(By.TAG_NAME, 'li')) > current_product_count
            )
            print(f"Số sản phẩm hiện tại: {len(driver.find_element(By.CLASS_NAME, 'listproduct').find_elements(By.TAG_NAME, 'li'))}")
            time.sleep(2)  # Đợi dữ liệu ổn định

            # Đặt lại số lần thử nếu nhấn thành công
            attempts = 0

        except Exception as e:
            # Kiểm tra xem nút "Xem thêm" có còn tồn tại không
            view_more_buttons = driver.find_elements(By.CLASS_NAME, 'see-more-btn')
            if not view_more_buttons:
                print("Xác nhận: Không còn nút 'Xem thêm', dừng lại.")
                break

            # Nếu nút vẫn còn, tăng số lần thử
            attempts += 1
            print(f"Lỗi khi nhấn nút 'Xem thêm' (lần {attempts}/{max_attempts}): {str(e)}")
            
            if attempts >= max_attempts:
                print(f"Đạt số lần thử tối đa ({max_attempts}), dừng lại.")
                break

            # Đợi lâu hơn và thử lại
            time.sleep(5)
            continue

    # Lấy nội dung HTML sau khi tải hết sản phẩm
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    # Tìm danh sách sản phẩm
    product_list = soup.find('ul', class_='listproduct')
    if not product_list:
        print("Không tìm thấy danh sách sản phẩm!")
        driver.quit()
        exit()

    # Lặp qua từng sản phẩm
    products = product_list.find_all('li', class_='item')
    print(f"Tìm thấy {len(products)} sản phẩm.")

    for index, product in enumerate(products):
        product_data = {}

        # Lấy thẻ <div class="main-contain"> chứa thông tin chính
        main_contain = product.find('a', class_='main-contain')
        if not main_contain:
            print(f"Bỏ qua sản phẩm {index + 1}: Không tìm thấy thẻ main-contain")
            continue

        # 1. Tên sản phẩm (trong thẻ <h3>)
        name_tag = main_contain.find('h3')
        if name_tag:
            new_model_tag = name_tag.find('span', class_='newModel')
            if new_model_tag:
                new_model_tag.decompose()
            product_data['name'] = name_tag.text.strip() if name_tag else "N/A"
        else:
            product_data['name'] = "N/A"
        
        RAM_ROM = main_contain.find('div', class_='item-compare')
        if RAM_ROM:
            RAM_ROM = RAM_ROM.find_all('span')
            if len(RAM_ROM) >= 2:
                product_data['RAM'] = RAM_ROM[0].text.strip() if RAM_ROM[0] else "N/A"
                product_data['ROM'] = RAM_ROM[1].text.strip() if RAM_ROM[1] else "N/A"
            else:
                product_data['RAM'] = "N/A"
                product_data['ROM'] = "N/A"

        data_brand = main_contain.get('data-brand')
        if data_brand:
            product_data['brand'] = data_brand.strip()
        else:
            product_data['brand'] = "N/A"
        
        color = main_contain.get('data-color')
        if color:
            product_data['color'] = color.strip()
        else:
            product_data['color'] = "N/A"

        data_cate = main_contain.get('data-cate')
        if data_cate:
            product_data['category'] = data_cate.strip()
        else:
            product_data['category'] = "N/A"

        data_price = main_contain.get('data-price')
        if data_price:
            product_data['price'] = data_price.strip()
        else:
            product_data['price'] = "N/A"
        
        price_old = main_contain.get('price-old')
        if price_old:
            product_data['price_old'] = price_old.strip()
        else:
            product_data['price_old'] = data_price
        
        percent = (float(product_data['price_old']) - float(product_data['price'])) / float(product_data['price_old']) * 100
        product_data['percent'] = str(int(percent))

        # 4. Thông tin đánh giá (trong thẻ <div class="rating_Compare has_compare has_quantity">)
        rating_div = product.find('div', class_='rating_Compare has_compare has_quantity')
        if rating_div:
            num_star = rating_div.find('b')
            if num_star:
                numstar = num_star.text.strip()
                if numstar == 'Chưa có đánh giá':
                    numstar = "0"
                product_data['rating'] = numstar
            else:
                product_data['rating'] = "0"

            toltal_sell = rating_div.find('span')
            if toltal_sell:
                totalcell = toltal_sell.text.split()
                n = len(totalcell)
                totalCell = totalcell[n - 1]
                # Chuyển đổi số lượng bán nếu có "K"
                if 'k' in totalCell:
                    # Loại bỏ "K", thay dấu "," thành ".", nhân với 1000
                    number = totalCell.replace('k', '').replace(',', '.')
                    totalCell = str(int(float(number) * 1000))
                product_data['sold'] = totalCell
            else:
                product_data['sold'] = "0"
        else:
            product_data['rating'] = "Chưa có đánh giá"
            product_data['sold'] = "N/A"
            if product_data['rating'] == 'Chưa có đánh giá':
                product_data['rating'] = "0"
            if product_data['sold'] == 'N/A':
                product_data['sold'] = "0"

        # Thêm dữ liệu sản phẩm vào danh sách
        all_products_data.append(product_data)
        print(f"Đã crawl sản phẩm {index + 1}: {product_data['name']}")

finally:
    # Đóng trình duyệt
    driver.quit()

# In dữ liệu và lưu vào file CSV
if all_products_data:
    print("\n---- Dữ liệu sản phẩm ----")
    for data in all_products_data:
        print(data)
    print("\n---- Kết thúc ----")

    # Lưu dữ liệu vào file CSV với thứ tự cột được chỉ định
    df = pd.DataFrame(all_products_data)
    df = df[['name', 'brand', 'category', 'color', 'price', 'price_old', 'RAM', 'ROM', 'percent', 'rating', 'sold']]
    df.to_csv('phone_data.csv', index=False, encoding='utf-8-sig')
    print("\nDữ liệu đã được lưu vào file 'phone_data.csv'")
else:
    print("Không có dữ liệu để lưu!")

Truy cập trang: https://www.thegioididong.com/laptop
Nhấn nút 'Xem thêm' để tải hết sản phẩm...
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 45
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 65
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 85
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 105
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 125
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 145
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 165
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 185
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 205
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 225
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 245
Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...
Số sản phẩm hiện tại: 265
Đã nhấn